In [ ]:
import regreg.api as rr
import numpy as np
%load_ext rpy2.ipython

In [ ]:
%%R -o X,Y
library(lars)
data(diabetes)
X = diabetes$x
Y = diabetes$y

In [ ]:
X = np.hstack([X, np.ones((X.shape[0],1))])
#betah = np.dot(np.linalg.pinv(X), Y)
#Yhat = np.dot(X,betah)
#smallest_l2_bound = np.linalg.norm(Y-Yhat)
#null_soln = Y-Y.mean()
#R = smallest_l2_bound / np.linalg.norm(null_soln)

In [ ]:
l2 = rr.l2norm.affine(X,-Y,bound=0.65*np.linalg.norm(Y))
l1 = rr.l1norm(X.shape[1], lagrange=1)
l2s = l2.smoothed(rr.identity_quadratic(1.e-12,0,0,0))
problem = rr.simple_problem(l2s, l1)

In [ ]:
primal_nesta, dual_nesta = rr.nesta(None, l1, l2)
np.linalg.norm(Y - np.dot(X, primal_nesta)) / np.linalg.norm(Y)

In [ ]:
l1_lagrange = rr.l1norm(X.shape[1],lagrange=np.fabs(primal_nesta).sum())
loss = rr.squared_error(X,Y, coef=2)
newsoln = rr.simple_problem(loss, l1_lagrange).solve()
np.linalg.norm(Y - np.dot(X,newsoln)) / np.linalg.norm(Y)

In [ ]:
transform, atom = l2.dual
primal_tfocs, dual_tfocs = rr.tfocs(l1, transform, atom)
np.linalg.norm(Y - np.dot(X, primal_tfocs)) / np.linalg.norm(Y)

In [ ]:
%timeit primal_tfocs, dual_tfocs = rr.tfocs(l1, transform, atom)

In [ ]:
%timeit primal_nesta, dual_nesta = rr.nesta(None, l1, l2)

In [ ]:
np.linalg.norm(primal_tfocs - primal_nesta) / (1+np.linalg.norm(primal_nesta))


In [ ]:
np.linalg.norm(dual_tfocs - dual_nesta) / (1+np.linalg.norm(dual_nesta))

## Noiseless case: minimimum L1 norm reconstruction

In [ ]:
n, p = 200, 5000
X = np.random.standard_normal((n, p))
l1 = rr.l1norm(p, lagrange=1)
beta = np.zeros(p)
beta[:10] = 10
Y = np.dot(X, beta)

constraint = rr.zero_constraint.affine(X,-Y)
transform, atom = constraint.dual
primal_tfocs, dual_tfocs = rr.tfocs(l1, transform, atom)




In [ ]:
np.linalg.norm(Y - np.dot(X, primal_tfocs)) / np.linalg.norm(Y)


In [ ]:
primal_tfocs[:20]